# Ejercicios para practicar LECTURA Y ESCRITURA DE FICHEROS
Descarga el archivo que se adjunta con la tarea (Musical_Instruments_5.json). Este archivo contiene opiniones de usuarios de Amazon sobre productos, en concreto instrumentos musicales. 

**1.**-  Debes escribir un programa que consiga los siguientes objetivos:

1. Importa le archivo de la actividad y convierte sus datos en un dataframe de pandas.
2. Quédate solo con las columnas "asin" (el código de producto) y "overall" (la nota que le dan en la opinión).
3. Crea un nuevo dataframe que muestre cada uno de los productos (su asin) con la puntuación media que ha obtenido en las opiniones y el número de opiniones que tiene.
4. Añade una columna al dataframe que muestre la moda.
5. Haz una función que haga lo que consideres oportuno para elegir un producto que comprar y devuelva el asin de la recomendación elegida. La función recibirá como parámetro el dataframe ya modificado y devolverá el asin del producto recomendado.

In [ ]:
import pandas as pd

df_instrumentos = pd.read_json('Musical_Instruments_5.json', lines=True)
df_instrumentos.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [ ]:
df_instrumentos_filtrado = df_instrumentos[['asin','overall']]

In [ ]:
df_instrumentos_filtrado.size

20522

In [ ]:
#Necesitamos un dataframe que quede así:
#asin        #media_overall #num_opiniones
#1384719342  #4.7             #5
#B00004Y2UT  #3.8             #22

In [ ]:
#Método 1, metiendo las futuras columnas en listas:


import numpy as np

num_asin = []
medias = []
num_opiniones = []

df_instrumentos_medias = pd.DataFrame(columns=['asin','media','opiniones'])
for index,row in df_instrumentos_filtrado.iterrows():
    #comprobar si asin esta en #num_asin
    #si no está, lo meto y si está no lo meto.
    if row['asin'] not in num_asin:
        num_asin.append(row['asin'])
        #filtrar el dataframe para coger todos los overalls que coinciden con el asin
        #meto la media de esos overalls en medias
        medias.append(df_instrumentos_filtrado.loc[df_instrumentos_filtrado['asin'] == row['asin']]['overall'].mean())
        #meto la longitud de la lista de overalls en num_opiniones
        num_opiniones.append(len(df_instrumentos_filtrado.loc[df_instrumentos_filtrado['asin'] == row['asin']]))

#Al acabar el for ya tengo las 3 listas terminadas, las puedo meter en el dataframe:
df_instrumentos_medias = pd.DataFrame({ 'asin' : num_asin,'overall' : medias,'reviews_number' : num_opiniones}) 
df_instrumentos_medias.head()


,asin,overall,reviews_number
0,1384719342,5.000000,5
1,B00004Y2UT,4.666667,6
2,B00005ML71,3.800000,5
3,B000068NSX,4.571429,7
4,B000068NTU,4.800000,5


In [ ]:
#Método 2: Hago el mismo dataframe, pero metiendo fila a fila:

import numpy as np

df_instrumentos_medias = pd.DataFrame(columns=['asin','media','opiniones'])
for index,row in df_instrumentos_filtrado.iterrows():
    #comprobar si asin esta en #num_asin
    #si no está, lo meto y si está no lo meto.
    if row['asin'] not in df_instrumentos_medias['asin'].to_list():
        #meto el asin en una variable
        codigo = row['asin']
        #meto la media en otra
        media = df_instrumentos_filtrado.loc[df_instrumentos_filtrado['asin'] == row['asin']]['overall'].mean()
        #y finalmente el número de opiniones
        opiniones = df_instrumentos_filtrado.loc[df_instrumentos_filtrado['asin'] == row['asin']].shape[0]
        #Ahora que tengo las 3 variables para meter una fila en el dataframe la meto.
        #Ojo, para meterla al final, uso como índice el shape del propio dataframe, pero solo la primera dimensión, es decir, el número de filas.
        #shape te devuelve 2 dimensiones y la segunda es el número de columnas.
        df_instrumentos_medias.loc[df_instrumentos_medias.shape[0]] = [codigo,media,opiniones]

df_instrumentos_medias.head()


,asin,media,opiniones
0,1384719342,5.000000,5
1,B00004Y2UT,4.666667,6
2,B00005ML71,3.800000,5
3,B000068NSX,4.571429,7
4,B000068NTU,4.800000,5


In [ ]:
# APARTADO 4: METER UNA COLUMNA CON LA MODA:
num_asin = []
modas = []
#volver a hacer el iterrows() de df_instrumentos_filtrado
for index,row in df_instrumentos_filtrado.iterrows():
    #comprobar que no he mirado ya este asin
    if row['asin'] not in num_asin:
        #guardamos que este ya está hecho
        num_asin.append(row['asin'])
        #sacar un dataframe con todas las filas de este producto
        df_temporal = df_instrumentos_filtrado.loc[df_instrumentos_filtrado['asin'] == row['asin']]
        #del dataframe me quedo solo con los overalls (me va a devolver un tipo pandas series)
        puntuaciones_producto = df_temporal['overall']
        #ahora saco la moda de esas puntuaciones y lo meto en la lista
        modas.append(int(puntuaciones_producto.mode().values[0]))

df_instrumentos_medias['modas'] = modas
df_instrumentos_medias.head(20)
        


,asin,media,opiniones,modas
0,1384719342,5.000000,5,5
1,B00004Y2UT,4.666667,6,5
2,B00005ML71,3.800000,5,5
3,B000068NSX,4.571429,7,5
4,B000068NTU,4.800000,5,5
5,B000068NVI,4.538462,13,5
6,B000068NW5,4.461538,52,5
7,B000068NZC,4.142857,7,5
8,B000068NZG,4.600000,5,5
9,B000068O1N,4.888889,9,5


In [ ]:
# APARTADO 5:
def recomienda (df_productos_preparado):
    #Primero busco el valor máximo de la columna opiniones y lo meto en maximo_num_reviews:
    maximo_num_reviews = df_productos_preparado['opiniones'].max()
    #Filtro el dataframe para quedarme con todas las filas que tienen al menos un 30% de maximo_num_reviews(0.3*maximo_num_reviews)
    df_productos_reviews_suficientes = df_productos_preparado.loc[df_productos_preparado['opiniones'] > maximo_num_reviews*0.3]
    #Ordeno el dataframe por la nota media de manera descendente. 
    df_productos_reviews_suficientes = df_productos_reviews_suficientes.sort_values(by='media', ascending=False)
    #Usando iloc filtro por posición el dataframe y me quedo con la posición 0, osea, el que más nota tenía.
    return(df_productos_reviews_suficientes.iloc[0]['asin'])

recomienda(df_instrumentos_medias)

'B0002D0CEO'